In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import time


# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin, clone

# XGBoost
from catboost import CatBoostClassifier


# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_tic import preprocessador_titanic


## 2. Dataload & Preprocessamento com joblib

In [2]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_Titanic_v1.2.joblib')
PP2=temp['preprocessador']

# # =====================================================
# # 📁 1. Leitura dos dados & Separação das bases
# # =====================================================

DATA_DIR = BASE / "data" / "raw"
X_train = pd.read_csv(DATA_DIR / "X_train_raw.csv").reset_index(drop=True)
X_val  = pd.read_csv(DATA_DIR / "X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_raw.csv").values.ravel()
y_val  = pd.read_csv(DATA_DIR / "y_test_raw.csv")


# =====================================================
# 📁 2. Modelo e pipeline
# =====================================================
pipe_CAT3 = joblib.load('modelo_CATBST_final_refine_v12.joblib')
pipe_CAT3.named_steps # informações sobre a pipeline

# model_base = CatBoostClassifier(loss_function='Logloss',eval_metric='Accuracy',random_state=42,
#                                verbose=False)

# pipe_CAT3 = pipe_models(model_base, PP2)

{'preprocess': preprocessador_titanic(),
 'model': <catboost.core.CatBoostClassifier at 0x786b450ae490>}

In [3]:
# =====================================================
# 3.Treino & Teste
# =====================================================
cv_scores = cross_val_score(pipe_CAT3 , X_train, y_train, cv=10)
pipe_CAT3.fit(X_train, y_train)
y_pred=pipe_CAT3.predict(X_val) #validação
#y_probs = pipe_CAT3.predict_proba(X_val)[:, 1]
#y_pred = (y_probs >= 0.48).astype(int)
print(f"{'='*70}")
print(f"🎯 CAT(refine) | cvscores : {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"{'='*70}")
print(f"📊 **Acurácia no Teste**: {accuracy_score(y_val, y_pred):.4f}")
print(f"\n📋 **Relatório de Classificação**:")
print(classification_report(y_val, y_pred))
cm=confusion_matrix(y_val, y_pred)
print(f"🎯 **Matriz de Confusão**:")
print(f"               Previsto 0   Previsto 1")
print(f"Real 0         {cm[0,0]:<11} {cm[0,1]:<11}")
print(f"Real 1         {cm[1,0]:<11} {cm[1,1]:<11}")
print(f"{'─'*70}")

🎯 CAT(refine) | cvscores : 0.8443 ± 0.0461
📊 **Acurácia no Teste**: 0.8172

📋 **Relatório de Classificação**:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       157
           1       0.82      0.72      0.77       111

    accuracy                           0.82       268
   macro avg       0.82      0.80      0.81       268
weighted avg       0.82      0.82      0.82       268

🎯 **Matriz de Confusão**:
               Previsto 0   Previsto 1
Real 0         139         18         
Real 1         31          80         
──────────────────────────────────────────────────────────────────────


## 3. Kaggle

In [5]:
# =====================================================
# Submissão Kaggle
# =====================================================
base = pd.read_csv("/home/akel/PycharmProjects/Kaggle/Titanic/data/raw/test.csv")

id_test = base["PassengerId"]

x_test=base.drop(columns='PassengerId')

# Previsão
y_pred_final=pipe_CAT3.predict(x_test)

#y_probs = pipe_CAT3.predict_proba(x_test)[:, 1]
#y_pred_final = (y_probs >= 0.48).astype(int)

#  DataFrame de submissão
submission = pd.DataFrame({
    "PassengerId": id_test,
    "Survived": y_pred_final
})

submission_path = ("/home/akel/PycharmProjects/Kaggle/Titanic/data/processed/submission_CAT_refine.v1.2.csv")
submission.to_csv(submission_path, index=False)
print("✅ Arquivo de submissão salvo com sucesso!")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))

✅ Arquivo de submissão salvo com sucesso!

#Processo finalizado em: 14:49:49
